<br>Dora Li
<br>CS315 
<br>April 26 th, 2024

# Merging Datasets (json and csv data)
**Table of Content**

1. [Function Definitions](#sec1)  
2. [Selecting Relevant Users](#sec2)  
3. [Sample Test](#sec3)

In [7]:
import json
import pandas as pd
from urllib.parse import urlparse
import matplotlib.pyplot as plt

<a id="sec1"></a>
## 1. Function Definitions

In [35]:
def getID(url):
    """
    Helper Function: Given an url string, returns the video id embedded in the url
    """
    parsed_url = urlparse(url)
    path_segments = parsed_url.path.split('/')
    return int(path_segments[3])

def processJson(jsond): #for Sec2Gr2 files
    """
    Helper Function: Given a json datafile name, return the file in dataframe format 
    """
    jsondf = jsond
    jsondf['video_id'] = jsondf['Link'].apply(getID)
    return jsondf

def processJson2(filename): #for Sec1Gr1 files
    """
    Helper Function: Given a json datafile name, return the file in dataframe format 
    """
    with open(filename, 'r') as file:
        jsondata = json.load(file)

    jsondict = {'Date':[],'Link':[],'video_id':[]}
    for video in jsondata['data']:
        jsondict['Link'].append(video[1])
        jsondict['video_id'].append(getID(video[1])) 
        jsondict['Date'].append(video[0])

    jsondf = pd.DataFrame(jsondict)
    return jsondf
    
    
    

<a id="sec2"></a>
## 2. Selecting Relevant Users

In [37]:
sec1files = ["Sec1Gr1_10824.json","Sec1Gr1_12345.json","Sec1Gr1_50405.json"]
sec1grp3file = "Sec1Gr3_11111.json"
sec2files = ["Sec2Gr2_26301.json","Sec2Gr2_33534.json","Sec2Gr2_38129.json","Sec2Gr2_69117.json"]

In [69]:
# look at section 2 users
for file in sec2files:
    print(file)
    jsond = pd.read_json(file)
    jsonfile = processJson(jsond)
    jsonfile['Date'] = pd.to_datetime(jsonfile['Date'])
    monthlycounts = jsonfile.groupby(jsonfile['Date'].dt.to_period('M')).size()
    print(monthlycounts)
    print()

Sec2Gr2_26301.json


FileNotFoundError: File Sec2Gr2_26301.json does not exist

In [30]:
# look at section1 users
for file in sec1files:
    print(file)
    jsonfile = processJson2(file)
    jsonfile['Date'] = pd.to_datetime(jsonfile['Date'])
    monthlycounts = jsonfile.groupby(jsonfile['Date'].dt.to_period('M')).size()
    print(monthlycounts)
    print()

Sec1Gr1_10824.json
Date
2023-11    14114
2023-12     2940
2024-02     9213
Freq: M, dtype: int64

Sec1Gr1_12345.json
Date
2023-09    2985
2023-11    1964
2023-12    5558
2024-01    5299
2024-02    4038
2024-03     777
Freq: M, dtype: int64

Sec1Gr1_50405.json
Date
2023-08    10053
2023-09     1044
2023-10     1540
2023-11     1003
2023-12     6047
2024-01     6295
2024-02      135
Freq: M, dtype: int64



<a id="sec3"></a>
## 3. Sample Test

In [63]:
#jsondf = processJson2("Sec1Gr1_50405.json")#sec1

jsond = pd.read_json("Sec2Gr2_26301.json")#sec2
jsondf = processJson(jsond)

jsondf.head()

,Date,Link,video_id
0,2024-03-06 19:44:41,https://www.tiktokv.com/share/video/7315561816...,7315561816673750318
1,2024-03-06 18:00:25,https://www.tiktokv.com/share/video/7315561816...,7315561816673750318
2,2024-03-06 17:58:36,https://www.tiktokv.com/share/video/7315561816...,7315561816673750318
3,2024-03-06 17:56:52,https://www.tiktokv.com/share/video/7315561816...,7315561816673750318
4,2024-03-06 17:46:03,https://www.tiktokv.com/share/video/7315561816...,7315561816673750318


In [66]:
csvdf = pd.read_csv("Sec2Gr2_26301.csv")
csvdf.head()

,video_id,video_timestamp,video_duration,video_locationcreated,suggested_words,video_diggcount,video_sharecount,video_commentcount,video_playcount,video_description,video_is_ad,video_stickers,author_username,author_name,author_followercount,author_followingcount,author_heartcount,author_videocount,author_diggcount,author_verified
0,7315561816673750318,2023-12-22T18:16:20,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3500000.0,39500.0,10800.0,18700000.0,Bonus flipnote for you,False,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False
1,7315561816673750318,2023-12-22T18:16:20,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3500000.0,39500.0,10800.0,18700000.0,Bonus flipnote for you,False,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False
2,7315561816673750318,2023-12-22T18:16:20,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3500000.0,39500.0,10800.0,18700000.0,Bonus flipnote for you,False,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False
3,7315561816673750318,2023-12-22T18:16:20,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3500000.0,39500.0,10800.0,18700000.0,Bonus flipnote for you,False,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False
4,7315561816673750318,2023-12-22T18:16:20,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3500000.0,39500.0,10800.0,18700000.0,Bonus flipnote for you,False,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False


In [67]:
result = jsondf.merge(csvdf, on=['video_id'])
result['Date'] = pd.to_datetime(result['Date'])
monthlycounts = result.groupby(result['Date'].dt.to_period('M')).size()
print(monthlycounts)
print()

Date
2023-09       1
2023-10      14
2023-11      15
2023-12      51
2024-01     128
2024-02    4757
2024-03    5764
Freq: M, dtype: int64



In [47]:
## look at the videos in the json
jsondf['Date'] = pd.to_datetime(jsondf['Date'])
monthlycounts = jsondf.groupby(jsondf['Date'].dt.to_period('M')).size()
print(monthlycounts)

Date
2023-09    5075
2023-10    5652
2023-11    7317
2023-12    8746
2024-01    9758
2024-02    3706
Freq: M, dtype: int64


In [68]:
# save the result to csv
result.to_csv("merged_26301.csv.csv")